# The fastest way to sort in python for an unknown-sized array

### 1. Defining different algorithms for comparason



#### I went ahead and implemented different sorting algorithms to show a clear comparison and recommend the best of them, I also took a look at the algorithm being currently used in the underlying "Sorted" function in python to get a sense of the markers for a good sorting algorithm and the source that I used for that is provided in the link 
https://realpython.com/sorting-algorithms-python/

In [48]:
from random import randint
from timeit import repeat 

#this code is intended to display the results
##the time is being calculated using the timeit library that calculates the execusion time for bits of code
def run_sorting_algorithm(algorithm, array):
    code=f"from __main__ import {algorithm}"\
    if algorithm !="sorted" else "" ## this is the built in sorting algorithm in python
    stmt=f"{algorithm}({array})"
    timers= repeat(setup=code, stmt=stmt, repeat=3, number =10000)## we will be repeating the excution of each 
    ## algorithm 10000 times and calculating the mean time of those execusions to return an answer and then
    ## we repeat that 3 times and return the minimum of those 3 avarages to get an accurate representation of the time taken
    print(f"minimum execusion time of the algorithm {algorithm} is: {min (timers)} s")
    

#### bubble sort is relatively slow, but I just implemented it for comparason

In [83]:
def bubble_sort(array):
    n=len(array)
    already_sorted=False
    for i in range(n):
        for j in range(n-i-1):
            if (array[j]>array[j+1]):
                x=array[j]
                array[j]=array[j+1]
                array[j+1]=x
                already_sorted=False
        if already_sorted: break
                
#     print (array)

#### insertion sort is also a slow algorithm when the data set is large in size, but it provides an efficient solution with small size sets and is a gate way for more complicated algorithms that are built upon it

In [38]:
def insertion_sort(array):
    n=len(array)
    for i in range (1,n):
        j=i-1
        key=array[j+1]
        while j>=0:
            if array[j]>key:
                array[j+1], array[j]=array[j],array[j+1]
                j-=1
            else: 
                array[j+1]=key
                
                break


#### merge sort is an efficient devide and conquer algorithm, it is stable in computation time, although it is beaten by algorithms like quick sort in cases of semi-sorted algorithms

In [116]:
def merge(arr1, arr2):
    n1=len(arr1)
    n2=len(arr2)
    if n1==0:
        
        return arr2
        
    
    if n2==0:
               return arr1
    result=[]
    ind1=ind2=0

    while ind1+ind2<n1+n2:
        if ind1>n1-1:
            result+=arr2[ind2:]
            break
        if ind2>n2-1:
            result+=arr1[ind1:] 
            break
        
        if arr1[ind1]<arr2[ind2]:
            result.append(arr1[ind1])
            ind1+=1
        else:
            result.append(arr2[ind2])
            ind2+=1
     
    return result



In [117]:
def merge_sort(arr):

        halfN=int(len(arr)/2)
        left=arr[:halfN]
        right=arr[halfN:]
#     print(left)
#     print(right)

        if len(left)>1:
             left=merge_sort(left)
        if len(right)>1:
             right=merge_sort(right)
                
        return merge(left, right)



#### quick sort is a very fast algorithm in many scenarios, it is also considered an in place sorting algorithm, meaning that it has great cache locality which makes it faster than merge-sort in many cases. However the algorithm is not stable and descends to a very poor performance in worst case scenarios depending on the index of the pivot element (which is in the best case the median of the array elements)

In [119]:
import random
def quick_sort(arr):
    if len(arr)==0:
        return []
    index=random.randint(0, len(arr)-1)
    pivot=arr[index]
    low=[]
    high=[]
    same=[]
    for i in range(len(arr)):
        if arr[i]<pivot:
            low.append(arr[i])
        else:
            if arr[i]>pivot:
            
                high.append(arr[i])
            else:
                same.append(arr[i])
                
    return quick_sort(low)+same+quick_sort(high)


### 2. my suggestion 
#### I will implement a variation of the timesort algorithm that is being currently used in the python built in "sorted" algorithm and explain why I chose this algorithm 

### * this algorithm creates a variation on insertion sort by adding 2 pointers (left, right) to the beginning and the end of the part being sorted in this way. This alteration allows it to use insertion sort directly on small size arrays and also allows it to take advantage of it in sorting chuncks of the array in a devide and conquer manner.

In [120]:
def insertion_sort2(array, left=0, right=None):
    n=len(array)
    if right== None:
        right =len(array)-1
        
    for i in range (left+1,right+1):
        j=i-1
        key=array[j+1]
        while j>=left:
            if array[j]>key:
                array[j+1], array[j]=array[j],array[j+1]
                j-=1
            else: 
                array[j+1]=key
                
                break
    return array


### * This sorting algorithm is a devide and conquer algorithm, it sorts small chuncks of the array using insertion sort and then merges them into the existing array progressively, merging larger sizes of the array with each iteration (the size is multiplied by 2 with each iteration)

In [121]:
def time_sort(arr):
    min_run=32 # the size of the chuncks devided here is defaulted to 32 but in the actual implementation in the python built 
    # function, this size is being calculated based on a variaty of factors and is always a power of 2
    
    n=len(arr)
    for i in range(0, n, min_run):
        insertion_sort2(arr, i, min(n-1,i+ min_run-1)) ## sort chunks of min_run size using insertion sort
        # and if the size is too small then the array is defaulted to be insertion sorted
    size=min_run
    
    while size<n:
        for i in range (0,n, size*2 ):
            midpoint=i+size-1
            end=min(n-1,i+size*2-1)
            if midpoint < end: # Only merge if there is something to merge
                merged=merge(arr[i:midpoint+1], arr[midpoint +1:end+1]) ## merging the chuncks 
                arr[i:end+1]=merged 
        size*=2 ## doubling the size of the chuncks being merged
    return arr

### 3. let's check the actual performance before reaching a suggestion

In [127]:

# firstly for a large sized random array

if __name__=="__main__":
    array_length=100
    array =[randint(0,100) for i in range(array_length)]
#     run_sorting_algorithm(algorithm="bubble_sort", array=array )
#     run_sorting_algorithm(algorithm="insertion_sort", array=array )
    run_sorting_algorithm(algorithm="merge_sort", array=array )
    run_sorting_algorithm(algorithm="quick_sort", array=array )
    run_sorting_algorithm(algorithm="time_sort", array=array )
    run_sorting_algorithm(algorithm="sorted", array=array )
    

minimum execusion time of the algorithm merge_sort is: 4.790492899948731 s
minimum execusion time of the algorithm quick_sort is: 3.3744631999870762 s
minimum execusion time of the algorithm time_sort is: 4.213390199933201 s
minimum execusion time of the algorithm sorted is: 0.044488899991847575 s


In [131]:
array= sorted(array)

In [134]:

# secondly for a sorted array

if __name__=="__main__":

    run_sorting_algorithm(algorithm="insertion_sort", array=array )
    run_sorting_algorithm(algorithm="merge_sort", array=array )
    run_sorting_algorithm(algorithm="quick_sort", array=array )
    run_sorting_algorithm(algorithm="time_sort", array=array )
    
    run_sorting_algorithm(algorithm="sorted", array=array )

minimum execusion time of the algorithm insertion_sort is: 0.336970700067468 s
minimum execusion time of the algorithm merge_sort is: 3.0517821999965236 s
minimum execusion time of the algorithm quick_sort is: 3.01374450000003 s
minimum execusion time of the algorithm time_sort is: 0.9637463000835851 s
minimum execusion time of the algorithm sorted is: 0.013452399987727404 s


### 4. final words

the general observation is that time_sort performs better than merge_sort for larger data sets, although it does not beat quick_sort in that regard. However if the array is sorted (or hypothetically, partially sorted), the performance of quick_sort descends significantelly and the performance of the quick sort algorithm remains impressively high, regardless of the size of the data set. 
So in that sense, time sort has the best of both worlds. It is a stable algorithm for large random arrays and it does better than merge sort in those regards. And yet it does not descend to bad performance in the worst case scenarios for other fast algorthims like quick sort. The reason for that is that it uses insertion for small chuncks and merge to conquer those small chuncks. 
So in terms of processor ticks, I generally advise the use of a variation of this algorithm. 
It still does not perform as the built in function, due to the lack of optimizations that are done for calculating the ideal size of minimum chuncks. However such optimizations help improve the performance significantely and are recomended. 
